In [37]:
import numpy as np
import cv2

def binarizar(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.medianBlur(frame, 9)
    ret, binarizado = cv2.threshold(blur, 4, 255, cv2.THRESH_BINARY)
    binarizado = cv2.dilate(binarizado, cv2.getStructuringElement(
        cv2.MORPH_ELLIPSE, (3,3)
    ), iterations = 2)
    return binarizado

def getCorners(frame, nx, ny):
    ret, corners = cv2.findChessboardCorners(frame, (nx, ny))
    print(corners)
    return ret, corners

def getPlanoDeFundo(frame):
    fgmask = cv2.createBackgroundSubtractorMOG2().apply(frame)
    fgmask = cv2.cvtColor(fgmask, cv2.COLOR_GRAY2BGR)
    cv2.imshow('fundo', fgmask)
    return fgmask

def desenharPontos(frame_inicial, ret, corners, nx, ny, objpoints):
    objp = np.zeros((nx*ny,2), np.float32)
    objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)
    if ret:
        pontos = np.int0(corners)
        for c in pontos:
            x,y = c.ravel()
            cv2.circle(frame_inicial, (x,y), 3, (0,0,255), -1)
        objpoints.append(objp)
        cv2.imshow('pontos', frame_inicial)

count = 0
nx, ny = 5,9
objpoints = []

video = cv2.VideoCapture('xadrez.mp4')
# Modelo de Fundo (tela vazia)
# Calcular a homografia com o 1° frame
_, frame_inicial = video.read()
gray = cv2.cvtColor(frame_inicial, cv2.COLOR_BGR2GRAY)

ret, corners = getCorners(gray, nx, ny)
desenharPontos(frame_inicial, ret, corners, nx, ny, objpoints)

# print(objpoints, corners)

# Calcular aqui a homografia
# h, mask = cv2.findHomography(corners, objpoints, cv2.RANSAC)
# print(h)

while video.isOpened():
    ok, frame = video.read()
    if frame is None:
        break

    count+=1
    if count <= 149:
        fundo = frame
        continue

    bolinha = cv2.subtract(frame, fundo)
    binarizada = binarizar(bolinha)
    
    # encontrar o menor ponto Y (branco)
    # desenhar no frame bolinha
    # multiplicação de H pelo menor ponto Y encontrado
    # matplotlib para plotar os pontos
    
    
    # cv2.imshow('frame', frame)
    # cv2.imshow('fundo', fundo)
    cv2.imshow('bolinha', bolinha)
    cv2.imshow('binarizado', binarizada)
    
    key = cv2.waitKey(30) & 0xff
    if key == ord('q'):
        break

video.release()
cv2.destroyAllWindows() 

None


None
